In [1]:
# Импортирование необходимых библиотек
import datetime
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [2]:
train_dataset = pd.read_csv('../data/backup/train_dataset.csv')
test_dataset = pd.read_csv('../data/backup/test_dataset.csv')

In [9]:
# #отрежем месяц для проверки через другой датасет
# secret_df = test_dataset.iloc[-720:]
# secret_df.to_csv("secret.csv",float_format='%.5f',mode='w',index=False)

In [3]:

# Сначала объедините колонки "date" и "time" в одну колонку "datetime"

def make_dt_col(row_date,row_time):
    return pd.to_datetime(row_date)+pd.to_timedelta(row_time,unit='h')
train_dataset['datetime']=train_dataset.apply(lambda x: make_dt_col(x['date'],x['time']),axis=1)

test_dataset['datetime']=test_dataset.apply(lambda x: make_dt_col(x['date'],x['time']),axis=1)

In [ ]:
# Установите "datetime" в качестве индекса
train_dataset.set_index('datetime', inplace=True)
test_dataset.set_index('datetime', inplace=True)

In [ ]:
# Если вам нужно убедиться, что индекс отсортирован, вы можете использовать
train_dataset.sort_index(inplace=True)
test_dataset.sort_index(inplace=True)
print(train_dataset.columns)
print(train_dataset.columns)

In [ ]:
# # Выберите только те столбцы, которые вы хотите преобразовать в числовой формат
# columns_to_convert = train_dataset.columns[:-2]  # Исключаем последние два столбца
# columns_to_convert = test_dataset.columns[:-2]  # Исключаем последние два столбца
# print(test_dataset.head())

In [ ]:
test_dataset['time'] = test_dataset['time'].apply(int)
test_dataset

In [ ]:

# Заменить все нечисловые значения на NaN
train_dataset = train_dataset.apply(pd.to_numeric, errors='coerce')
test_dataset = test_dataset.apply(pd.to_numeric, errors='coerce')


In [ ]:

# Продолжайте с преобразованием в числовой формат и дальнейшей обработкой данных
train_dataset = train_dataset.astype('float32')
columns_to_convert = train_dataset.columns[:-2]

test_dataset = test_dataset.astype('float32')
columns_to_convert = test_dataset.columns[:-2]

In [ ]:
# Заполнение пропущенных значений средними значениями
train_dataset['temp'].fillna(train_dataset['temp'].mean(), inplace=True)
test_dataset['temp'].fillna(test_dataset['temp'].mean(), inplace=True)
train_dataset['temp_pred'].fillna(train_dataset['temp_pred'].mean(), inplace=True)
test_dataset['temp_pred'].fillna(test_dataset['temp_pred'].mean(), inplace=True)

In [ ]:
# Просмотр данных
print("Тренировочные данные:")
print(train_dataset.head())

print("Тестовые данные:")
print(test_dataset.head())


In [ ]:
# Теперь выберите числовые признаки для обучения
X_train = train_dataset[['temp', 'time','temp_pred']]
y_train = train_dataset['target']

X_test = test_dataset[['temp', 'time','temp_pred']]
y_test = test_dataset['target']

Создание и обучение моделей

In [ ]:
# # Линейная регрессия
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

In [ ]:
# # Случайный лес
model_rf = RandomForestRegressor(n_estimators=100, random_state=0)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

In [ ]:
# MAPE (Mean Absolute Percentage Error)
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_lr = calculate_mape(y_test, y_pred_lr)
mape_rf = calculate_mape(y_test, y_pred_rf)

print("MAPE (Linear Regression): {:.2f}%".format(mape_lr))
print("MAPE (Random Forest): {:.2f}%".format(mape_rf))

In [ ]:
# R2 score
r2_lr = r2_score(y_test, y_pred_lr)
r2_rf = r2_score(y_test, y_pred_rf)
print("R2 Score (Linear Regression): {:.2f}".format(r2_lr))
print("R2 Score (Random Forest): {:.2f}".format(r2_rf))

In [ ]:
# Вычисление MAE
def show_mae(y1,y2):
    print("Mean Absolute Error (MAE):",mean_absolute_error(y1,y2))
print("По модели линейной регрессии:")
show_mae(y_test,y_pred_lr)
print("По модели рандом форест:")
show_mae(y_test,y_pred_rf)

выбираем модель рандом форест по лучшим показателям

In [ ]:

y_pred = y_pred_rf
model = model_rf
# И выводим предсказания
print(y_pred)

Сохраняем модель в файл через joblib

In [ ]:

with open("../model/trained.joblib","wb") as dump_file:
    dump(model,dump_file)

In [ ]:
# Создаем график
plt.figure(figsize=(20, 12))
# Рисуем точки с исходными данными из тестового набора данных
plt.scatter(X_test['temp'], y_test, label='Исходные данные', color='blue')
# Рисуем предсказания модели
plt.scatter(X_test['temp'], y_pred, label='Предсказания', color='red', marker='x')
# Добавляем легенду
plt.legend(loc='upper left')
# Добавляем подписи к осям
plt.xlabel('Температура')
plt.ylabel('Потребление')
# Отображаем график
plt.show()